<a href="https://colab.research.google.com/github/matheusvazdata/dados-lab/blob/main/Nested_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🎯 Funções Aninhadas e Engenharia de Dados

### 📌 Cenário 1: Validação e transformação de dados de clientes
Há um DataFrame com dados de usuários e será preciso limpar os nomes e validar os e-mails.

In [1]:
import pandas as pd

# Dicionário com os dados de exemplo
dados = {
    'nome': [' Alice ', 'bob', ' CAROL ', 'daniel', 'MATHEUS'],
    'email': ['alice@email.com', 'bob#email.com', 'carol@email.com', '', 'matheusvaz@email.com.br']
}

# Transformação para um Dataframe
df = pd.DataFrame(dados)

✅ Função com função aninhada

In [2]:
def limpar_validar(df):
    def limpar_nome(nome):
        return nome.strip().capitalize()

    def validar_email(email):
        return '@' in email and '.' in email

    df['nome_limpo'] = df['nome'].apply(limpar_nome)
    df['email_valido'] = df['email'].apply(validar_email)
    return df

In [3]:
# Visualização após aplicação da função aninhada
df_limpo = limpar_validar(df)
display(df_limpo)

,nome,email,nome_limpo,email_valido
0,Alice,alice@email.com,Alice,True
1,bob,bob#email.com,Bob,False
2,CAROL,carol@email.com,Carol,True
3,daniel,,Daniel,False
4,MATHEUS,matheusvaz@email.com.br,Matheus,True


In [4]:
# Filtrando os usuários com e-mails válidos e com seus nomes tratados
df_limpo = limpar_validar(df)
display(
    (df_limpo[df_limpo['email_valido'] == True]) \
    .loc[:, ['nome_limpo', 'email']] \
    .reset_index(drop=True)
)

,nome_limpo,email
0,Alice,alice@email.com
1,Carol,carol@email.com
2,Matheus,matheusvaz@email.com.br


As **funções aninhadas** `limpar_nome` e `validar_email` estão definidas **dentro da função principal** `limpar_validar`. Isso significa que elas só existem no contexto de execução dessa função. Essa abordagem traz três vantagens importantes para quem está migrando para Engenharia de Dados:

1. **Organização do código**: ao agrupar as funções auxiliares onde elas realmente são utilizadas, o código fica mais **modular e legível**.
2. **Escopo controlado**: as funções internas **não ficam disponíveis no escopo global**, evitando conflitos com outras partes do sistema.
3. **Segurança e reutilização pontual**: essas funções são usadas **apenas dentro da `limpar_validar`**, o que garante que **não sejam chamadas por engano** fora do seu propósito.

> Esse padrão é especialmente útil em pipelines de dados, onde é comum encapsular lógicas específicas de transformação ou validação em etapas bem definidas.

### 📌 Cenário 2: Cálculo de métricas com base em dados temporários (dentro de uma função)

In [5]:
# Escopo e a Regra LEGB
def gerar_relatorio():
    total_registros = 0  # Variável no escopo da função externa (Enclosing)

    def registrar(qtd):
        nonlocal total_registros  # Refere-se à variável da função externa
        total_registros += qtd    # Atualiza o valor acumulado
        return total_registros    # Retorna o total até o momento

    print(registrar(5))   # 5: adiciona 5 ao total_registros (inicialmente 0)
    print(registrar(10))  # 15: adiciona 10 ao valor atual (5 + 10)
    return total_registros

In [6]:
resultado = gerar_relatorio()
print("Total final:", resultado)  # Total final: 15

5
15
Total final: 15


Neste exemplo, temos uma **função aninhada** `registrar()` dentro da função `gerar_relatorio()`. A variável `total_registros` está no escopo **envolvente (enclosing)** da função `registrar`.

Ao usar a palavra-chave `nonlocal`, permitimos que a função interna **acesse e modifique** essa variável da função externa — algo que não seria possível com variáveis locais comuns.

### 🧠 LEGB: Como o Python resolve nomes de variáveis

- **L**ocal → dentro da função atual (`registrar`)
- **E**nclosing → dentro da função `gerar_relatorio`
- **G**lobal → no módulo (arquivo) atual
- **B**uilt-in → funções e nomes padrão do Python

### 🛠️ Aplicação real em Engenharia de Dados

Essa lógica pode ser útil, por exemplo, quando você está dentro de uma função que processa um conjunto de dados (como uma lista ou DataFrame) e quer manter **um contador interno de registros válidos**, sem usar variáveis globais.

Imagine um pipeline de validação que percorre registros e conta quantos passaram nos critérios. Você poderia encapsular esse contador com `nonlocal` dentro de uma função externa que orquestra as validações.

### 🎯 Aplicando tudo em uma mini-pipeline

In [7]:
def pipeline_dados(dados, fator_ajuste=1.0, **metadados):
    def limpar_nome(nome):
        return nome.strip().title()

    def aplicar_ajuste(valor):
        return valor * fator_ajuste

    dados['nome'] = dados['nome'].apply(limpar_nome)
    dados['receita_ajustada'] = dados['receita'].apply(aplicar_ajuste)

    print("\n[LOG] Metadados de execução:")
    for k, v in metadados.items():
        print(f"{k}: {v}")

    return f'\n{dados}'

In [8]:
df_exemplo = pd.DataFrame({
    'nome': [' joana ', 'MARCOS', ' Ana '],
    'receita': [1000, 1500, 1300]
})

resultado = pipeline_dados(df_exemplo, fator_ajuste=1.1, usuario='jr_data_eng', etapa='ajuste_financeiro')
print(resultado)


[LOG] Metadados de execução:
usuario: jr_data_eng
etapa: ajuste_financeiro

     nome  receita  receita_ajustada
0   Joana     1000            1100.0
1  Marcos     1500            1650.0
2     Ana     1300            1430.0


### 🧠 Conclusão
Esses conceitos são poderosíssimos para quem está em transição de carreira, especialmente quando:

- Você precisa organizar etapas de um pipeline ETL.

- Está construindo funções que serão reutilizadas em análises e dashboards.

- Precisa lidar com validação, transformação e logging de forma limpa e funcional.